In [7]:
import openai
import json
import os
from dotenv import load_dotenv 

load_dotenv()
openai.api_key = os.getenv("openai.api_key")



In [2]:
ai_response = []
messages = []
llm = None

In [3]:
def _openai(input) -> str:
    response = openai.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=input,
    )
    ai_response.append(response.choices[0].message.content)
    return response.choices[0].message.content

#####LOCAL AI WITH SCREENSHOT

In [25]:
import base64
import requests


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')



def _openai_local(prompt: str, image: str):
    # Path to your image
    image_path = image

    # Getting the base64 string
    base64_image = encode_image(image_path)

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {openai.api_key}"
    }

    payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": prompt
            },
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "low"
            }
            }
        ]
        }
    ],
    "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    return (response.json())

In [26]:
image = "screenshot.png"
prompt = "Analyze the provided 512x512 pixel image for a CAPTCHA button, typically used to confirm a user is not a bot. Identify the button's central point and provide its coordinates in terms of X and Y axis within this image size. The coordinates should help locate the button's center, which needs to be pressed to pass the CAPTCHA."

response = _openai_local(prompt, image)

print(response)

{'id': 'chatcmpl-98lnfvFsoqFVCF6X1HubT6h7i6Tl1', 'object': 'chat.completion', 'created': 1711877395, 'model': 'gpt-4-1106-vision-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "I'm sorry, but I can't assist with that request."}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 162, 'completion_tokens': 13, 'total_tokens': 175}, 'system_fingerprint': None}


In [22]:
print(response.json)

<bound method Response.json of <Response [200]>>


In [4]:
def request(image_url: str, prompt:str)-> str:
        messages.append ({
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                "type": "image_url",
                "image_url": {
                    "url": image_url,
                },
                },
            ],
            }
            )
        return _openai(messages)
    

In [5]:
def remove_markdown_code_block_markers(text):
    # Define the markers
    start_marker = "```json"
    end_marker = "```"

    # Remove the start marker
    if text.startswith(start_marker):
        text = text[len(start_marker):]

    # Remove the end marker
    if text.endswith(end_marker):
        text = text[:-len(end_marker)]

    return text.strip()



In [6]:
def turn_to_json(prompt):


  response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": prompt}
    ]
  )
  print(response.choices[0].message.content)
  to_safe = response.choices[0].message.content
  return to_safe


In [ ]:
extratcion_prompt = """Please analyze the image and generate a JSON object describing the social situation depicted. 
Your response should include details about the individuals present, their activities, and identifiable characteristics. 
Include a timestamp and format your response as a JSON object with the following attributes for each person: ID, position, activity, age, skincolor, clothing, posture, 
facial_expression, observable_features. Please use the following JSON structure as a template and dont return anything else, no disclaimers: 
{
  "timestamp": "current time", 
  "persons": [
    {
      "ID": "",
      "position": "",
      "activity": "",
      "age": "",
      "sex": "",
      "mood": "",
      "skincolor": "",
      "clothing": {
        "upper-body": "",
        "lower-body": "",
        "headwear": ""
      },
      "posture": "",
      "facial_expression": "",
      "observable_features": {
        "hairstyle": "",
        "haircolor": "",
        "accessories": ""
      }
    }
  ]
}"""

In [7]:
def extract(image_url: str) -> str:
    return request(
        image_url=image_url,
        prompt = 

    )

In [131]:


def recognise(image_url:str, person_desciption:str)->str: 
    return request(
        image_url= image_url,
        prompt= f"""Try to identify a specific person using this description: {person_desciption} in the new image. 
        Focus on matching features such as cothing, hairstyle, facial expressions, posture, and any unique identifiers 
        that were previously noted. Describe how each observable feature in the new image aligns with the information in the 
        JSON object to confirm the identity of the person. If the person is present, provide additional information about that person
        and try to conclude what has happened between the two pictures given the time difference between the pictures is 5 seconds. 
        If the Person is not present. Simply respone with 'the person does not seem to be in the picture' Answer as text"""
    )



****USAGE

In [132]:
extraction = extract("screenshot.png")



In [133]:
extraction = remove_markdown_code_block_markers(extraction)
print(extraction)

file_path = 'Extraction.json'

# Write the data to the file in JSON format
with open(file_path, 'a') as file:
    json.dump(extraction, file, indent=4)

{
  "timestamp": "2023-04-12T12:00:00Z", 
  "persons": [
    {
      "ID": "Person1",
      "position": "left",
      "activity": "posing for a photo",
      "age": "adult",
      "skincolor": "light",
      "clothing": {
        "upper-body": "light-colored shirt",
        "lower-body": "dark pants",
        "headwear": "sunglasses"
      },
      "posture": "standing",
      "facial_expression": "smiling",
      "observable_features": {
        "hairstyle": "short",
        "haircolor": "dark",
        "accessories": "watch on left wrist"
      }
    },
    {
      "ID": "Person2",
      "position": "center-left",
      "activity": "posing for a photo",
      "age": "child",
      "skincolor": "light",
      "clothing": {
        "upper-body": "red traditional outfit",
        "lower-body": "red traditional outfit",
        "headwear": "none"
      },
      "posture": "standing",
      "facial_expression": "smiling",
      "observable_features": {
        "hairstyle": "short",
      

In [136]:
print(ai_response)

['```json\n{\n  "timestamp": "2023-04-12T12:00:00Z", \n  "persons": [\n    {\n      "ID": "Person1",\n      "position": "left",\n      "activity": "posing for a photo",\n      "age": "adult",\n      "skincolor": "light",\n      "clothing": {\n        "upper-body": "light-colored shirt",\n        "lower-body": "dark pants",\n        "headwear": "sunglasses"\n      },\n      "posture": "standing",\n      "facial_expression": "smiling",\n      "observable_features": {\n        "hairstyle": "short",\n        "haircolor": "dark",\n        "accessories": "watch on left wrist"\n      }\n    },\n    {\n      "ID": "Person2",\n      "position": "center-left",\n      "activity": "posing for a photo",\n      "age": "child",\n      "skincolor": "light",\n      "clothing": {\n        "upper-body": "red traditional outfit",\n        "lower-body": "red traditional outfit",\n        "headwear": "none"\n      },\n      "posture": "standing",\n      "facial_expression": "smiling",\n      "observable_fea

In [88]:
prompt = f"Return the 'content' in this: {extraction} as a JSON"


In [89]:
to_safe = turn_to_json(prompt)

{
    "persons": [
        {
            "position": "left",
            "activity": "holding a child",
            "age": "adult",
            "skincolor": "brown",
            "clothing": {
                "upper-body": "grey raglan shirt with black sleeves",
                "lower-body": "dark blue jeans",
                "headwear": ""
            },
            "posture": "sitting",
            "facial_expression": "smiling",
            "observable_features": {
                "hairstyle": "short",
                "haircolor": "black",
                "accessories": "glasses"
            }
        },
        {
            "position": "center",
            "activity": "being held",
            "age": "infant",
            "skincolor": "brown",
            "clothing": {
                "upper-body": "white shirt with blue dots",
                "lower-body": "blue shorts",
                "headwear": ""
            },
            "posture": "sitting on lap",
            "facial_exp

In [134]:
img = "https://images.unsplash.com/photo-1655185497004-f3018eab9cb8?w=400&auto=format&fit=crop&q=60&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxjb2xsZWN0aW9uLXBhZ2V8MzR8MzgyOTE2OXx8ZW58MHx8fHx8"

desc = """Male, short hair, white skin, could be asian in apperance, usually well dressed"""

recognision = recognise (img, desc)

In [135]:
print(recognision)

Based on the description provided, we are looking for a male individual with short hair, white skin potentially possessing an Asian appearance, who is characterized as being well-dressed.

The new image displays a group of four individuals walking down a street. There is one male adult who matches the description:

- His skin tone could be described as white, and he may have an Asian appearance.
- This individual has short hair.
- He is well-dressed, sporting a light-colored, long-sleeve shirt and dark trousers.

Observable features that confirm this match include:
- Clothing: He is wearing the same light-colored long-sleeve shirt and dark trousers as mentioned in the description, indicating a well-dressed appearance.
- Hairstyle: The short hair matches the description.
- Facial expression and posture: Since the time difference between the images is only five seconds, the facial expression and posture are likely to be similar.

Given the consistency of these features with the descripti